In [ ]:
import tensorflow as tf

from MAML_enhanced import MAMLEnhanced
import sys

import numpy as np
import matplotlib.pyplot as plt
import os
import logging

dtype="float32"
tf.keras.backend.set_floatx(dtype)
print('Python version: ', sys.version)
print('TensorFlow version: ', tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('GPU found at: {}'.format(device_name))
plt.style.use('seaborn')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


Python version:  3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
TensorFlow version:  2.4.1
GPU found at: /device:GPU:0
1 Physical GPUs, 1 Logical GPUs


In [2]:
def generate_sinusoid(K):
    ranges = [-5, 5]
    amplitude = np.random.uniform(0.1, 5.0)
    phase = np.random.uniform(0, np.pi)
    xs = np.random.uniform(ranges[0], ranges[1], K)
    ys = amplitude * np.sin(xs - phase)
    xt = np.random.uniform(ranges[0], ranges[1], K)
    yt = amplitude * np.sin(xt - phase)
    return tf.convert_to_tensor(xs[:, None], dtype=dtype), tf.convert_to_tensor(ys[:, None], dtype=dtype), tf.convert_to_tensor(xt[:, None], dtype=dtype), tf.convert_to_tensor(yt[:, None], dtype=dtype)

In [3]:
def split_ds(ds):
    xs = ds[0]
    ys = ds[1]
    xt = ds[2]
    yt = ds[3]
    return xs, ys, xt, yt

In [4]:
AUTOTUNE = tf.data.AUTOTUNE
def create_sinusoid_ds(K, batch_size, train=True):
    if train:
        size = 10000
    else:
        size = 600
    all_ds = []
    for _ in range(size):
        xs, ys, xt, yt = generate_sinusoid(K)
        all_ds.append([xs,ys,xt,yt])
    ds = tf.data.Dataset.from_tensor_slices(all_ds)
    ds = ds.map(split_ds, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    return ds

In [6]:
fshot = 10
meta_batch_size = 16
ds =create_sinusoid_ds(fshot, meta_batch_size)

In [7]:
inp = next(iter(ds))

In [9]:
def SineModel():
    return tf.keras.models.Sequential([
    tf.keras.layers.Dense(40, input_shape=(1,)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(40),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(1, activation='linear'),
    ])

In [ ]:
model = SineModel()
maml_trainer = MAMLEnhanced(model)
maml_trainer.meta_train(ds, 10)

Update 1 : Outer loss: 3.6224489212036133   time: 20.430495999999998
Update 2 : Outer loss: 4.097020149230957   time: 0.4999551000000011
Update 3 : Outer loss: 3.2643582820892334   time: 0.4802925000000009
Update 4 : Outer loss: 3.0888426303863525   time: 0.4827432000000016
Update 5 : Outer loss: 3.426053285598755   time: 0.476768100000001
Update 6 : Outer loss: 4.315478801727295   time: 0.48846719999999877
Update 7 : Outer loss: 2.6361336708068848   time: 0.4903018999999986
Update 8 : Outer loss: 4.834048271179199   time: 0.4857927000000011
Update 9 : Outer loss: 4.512946128845215   time: 0.48510160000000013
Update 10 : Outer loss: 3.4239449501037598   time: 0.4859591999999999
Update 11 : Outer loss: 2.7910234928131104   time: 0.4708021000000002
Update 12 : Outer loss: 5.546334266662598   time: 0.48002389999999906
Update 13 : Outer loss: 4.082603454589844   time: 0.48581620000000214
Update 14 : Outer loss: 1.929172158241272   time: 0.4965918000000009
Update 15 : Outer loss: 4.31582355

Update 119 : Outer loss: 3.022733688354492   time: 0.4819711999999896
Update 120 : Outer loss: 4.789353370666504   time: 0.48003469999999027
Update 121 : Outer loss: 1.9695358276367188   time: 0.49516819999999484
Update 122 : Outer loss: 4.138032913208008   time: 0.4931866999999954
Update 123 : Outer loss: 3.7867274284362793   time: 0.4818348999999955
Update 124 : Outer loss: 3.499253749847412   time: 0.4820171000000073
Update 125 : Outer loss: 2.8596935272216797   time: 0.4917956000000032
Update 126 : Outer loss: 3.371828079223633   time: 0.48242979999999136
Update 127 : Outer loss: 2.4837899208068848   time: 0.4810695999999979
Update 128 : Outer loss: 2.882873058319092   time: 0.4908300999999966
Update 129 : Outer loss: 3.014997959136963   time: 0.48661780000000476
Update 130 : Outer loss: 3.2492527961730957   time: 0.5020291000000014
Update 131 : Outer loss: 2.42549991607666   time: 0.4910509999999988
Update 132 : Outer loss: 3.0896377563476562   time: 0.4744116000000105
Update 133 

Update 235 : Outer loss: 0.9079817533493042   time: 0.4650551000000007
Update 236 : Outer loss: 1.0336980819702148   time: 0.4851411000000212
Update 237 : Outer loss: 1.5464059114456177   time: 0.4869722999999908
Update 238 : Outer loss: 1.7172515392303467   time: 0.4795514000000196
Update 239 : Outer loss: 1.3096694946289062   time: 0.4774400999999955
Update 240 : Outer loss: 1.4599217176437378   time: 0.46664599999999723
Update 241 : Outer loss: 0.8778021335601807   time: 0.4700852999999938
Update 242 : Outer loss: 1.0301796197891235   time: 0.46824179999998705
Update 243 : Outer loss: 1.689286231994629   time: 0.475303700000012
Update 244 : Outer loss: 1.4784202575683594   time: 0.47751479999999447
Update 245 : Outer loss: 0.9984415769577026   time: 0.4890129999999999
Update 246 : Outer loss: 1.0355985164642334   time: 0.4697479999999814
Update 247 : Outer loss: 0.7727351188659668   time: 0.4684182000000021
Update 248 : Outer loss: 0.7135959267616272   time: 0.47427290000001676
Upda

Update 350 : Outer loss: 0.5028501749038696   time: 0.47869129999997995
Update 351 : Outer loss: 0.701081395149231   time: 0.4817079999999976
Update 352 : Outer loss: 0.42179393768310547   time: 0.4590350000000001
Update 353 : Outer loss: 0.9521581530570984   time: 0.45889259999998444
Update 354 : Outer loss: 0.7976686954498291   time: 0.47818359999999416
Update 355 : Outer loss: 0.8138763904571533   time: 0.4849160999999924
Update 356 : Outer loss: 0.4667443037033081   time: 0.46718429999998534
Update 357 : Outer loss: 0.6655489206314087   time: 0.4635776000000078
Update 358 : Outer loss: 0.3753248155117035   time: 0.4552127999999982
Update 359 : Outer loss: 0.5435025691986084   time: 0.45866340000000605
Update 360 : Outer loss: 0.5458595752716064   time: 0.476552799999979
Update 361 : Outer loss: 0.5152627229690552   time: 0.46253920000000903
Update 362 : Outer loss: 0.9777649641036987   time: 0.4503216999999893
Update 363 : Outer loss: 0.35151416063308716   time: 0.46657600000000343

Update 465 : Outer loss: 0.6206789016723633   time: 0.48239669999998114
Update 466 : Outer loss: 0.5468640923500061   time: 0.4800449000000242
Update 467 : Outer loss: 0.3667852282524109   time: 0.4773620000000278
Update 468 : Outer loss: 0.40886878967285156   time: 0.47699899999997797
Update 469 : Outer loss: 0.8194085955619812   time: 0.46828990000000204
Update 470 : Outer loss: 0.29180651903152466   time: 0.4639297999999883
Update 471 : Outer loss: 0.2577342092990875   time: 0.46643219999998564
Update 472 : Outer loss: 1.178871989250183   time: 0.47368899999997893
Update 473 : Outer loss: 0.39024919271469116   time: 0.45049839999998653
Update 474 : Outer loss: 0.4277106523513794   time: 0.46879069999999956
Update 475 : Outer loss: 0.29377496242523193   time: 0.4654075999999918
Update 476 : Outer loss: 0.24116450548171997   time: 0.4758099999999956
Update 477 : Outer loss: 0.7535016536712646   time: 0.4703644999999881
Update 478 : Outer loss: 0.6127258539199829   time: 0.453727399999

...

Update 4512 : Outer loss: 0.04649655893445015   time: 0.45280860000002576
Update 4513 : Outer loss: 0.03456714376807213   time: 0.4655155999998897
Update 4514 : Outer loss: 0.03388452157378197   time: 0.47209090000023934
Update 4515 : Outer loss: 0.033685121685266495   time: 0.45993799999996554
Update 4516 : Outer loss: 0.058252885937690735   time: 0.4682652000001326
Update 4517 : Outer loss: 0.023705048486590385   time: 0.4575435000001562
Update 4518 : Outer loss: 0.056504398584365845   time: 0.47229280000010476
Update 4519 : Outer loss: 0.0445827953517437   time: 0.4798929000003227
Update 4520 : Outer loss: 0.12763720750808716   time: 0.4764922999997907
Update 4521 : Outer loss: 0.028056617826223373   time: 0.48293550000016694
Update 4522 : Outer loss: 0.03616679459810257   time: 0.45688609999979235
Update 4523 : Outer loss: 0.061561957001686096   time: 0.4621415000001434
Update 4524 : Outer loss: 0.03171350061893463   time: 0.45797399999992194
Update 4525 : Outer loss: 0.02710200473

Update 4623 : Outer loss: 0.026005655527114868   time: 0.46645600000010745
Update 4624 : Outer loss: 0.013187702745199203   time: 0.4727426999997988
Update 4625 : Outer loss: 0.03185327351093292   time: 0.4809791000002406
Update 4626 : Outer loss: 0.027523860335350037   time: 0.47107240000013917
Update 4627 : Outer loss: 0.02761860005557537   time: 0.481357300000127
Update 4628 : Outer loss: 0.02104056440293789   time: 0.4722270000002027
Update 4629 : Outer loss: 0.03195910155773163   time: 0.46197410000013406
Update 4630 : Outer loss: 0.04443978890776634   time: 0.46985110000014174
Update 4631 : Outer loss: 0.033952657133340836   time: 0.4927906999996594
Update 4632 : Outer loss: 0.02587641403079033   time: 0.47023330000001806
Update 4633 : Outer loss: 0.016133852303028107   time: 0.4643047999998089
Update 4634 : Outer loss: 0.048046283423900604   time: 0.47693359999993845
Update 4635 : Outer loss: 0.041304685175418854   time: 0.4697138999999879
Update 4636 : Outer loss: 0.03336009383

Update 4734 : Outer loss: 0.03444546088576317   time: 0.4591877000002569
Update 4735 : Outer loss: 0.02995353564620018   time: 0.47379129999990255
Update 4736 : Outer loss: 0.032012540847063065   time: 0.46331740000005084
Update 4737 : Outer loss: 0.03732345253229141   time: 0.465947000000142
Update 4738 : Outer loss: 0.05325223132967949   time: 0.46245330000010654
Update 4739 : Outer loss: 0.02660193294286728   time: 0.4670774999999594
Update 4740 : Outer loss: 0.059358496218919754   time: 0.474902800000109
Update 4741 : Outer loss: 0.022818461060523987   time: 0.4696763000001738
Update 4742 : Outer loss: 0.04542239382863045   time: 0.4845490000002428
Update 4743 : Outer loss: 0.05519357696175575   time: 0.48490780000020095
Update 4744 : Outer loss: 0.035668570548295975   time: 0.4858387999997831
Update 4745 : Outer loss: 0.02564838156104088   time: 0.48094339999988733
Update 4746 : Outer loss: 0.028124362230300903   time: 0.4730149000001802
Update 4747 : Outer loss: 0.026214117184281

Update 4846 : Outer loss: 0.01917693391442299   time: 0.46302650000006906
Update 4847 : Outer loss: 0.022875722497701645   time: 0.45214379999970333
Update 4848 : Outer loss: 0.11829374730587006   time: 0.4666490999998132
Update 4849 : Outer loss: 0.0516931526362896   time: 0.46420579999994516
Update 4850 : Outer loss: 0.014785412698984146   time: 0.4596769999998287
Update 4851 : Outer loss: 0.019797027111053467   time: 0.47035790000018096
Update 4852 : Outer loss: 0.05401317775249481   time: 0.463235899999745
Update 4853 : Outer loss: 0.05254129320383072   time: 0.46089149999988877
Update 4854 : Outer loss: 0.07967793196439743   time: 0.45150299999977506
Update 4855 : Outer loss: 0.030821962282061577   time: 0.4560201999997844
Update 4856 : Outer loss: 0.04199690744280815   time: 0.46254110000018045
Update 4857 : Outer loss: 0.01963205635547638   time: 0.4549873999999363
Update 4858 : Outer loss: 0.02397758886218071   time: 0.4517501000000266
Update 4859 : Outer loss: 0.05073457956314

Update 4958 : Outer loss: 0.045018330216407776   time: 0.4794068999999581
Update 4959 : Outer loss: 0.03195328265428543   time: 0.4822312999999667
Update 4960 : Outer loss: 0.019154395908117294   time: 0.4725091000000248
Update 4961 : Outer loss: 0.023045454174280167   time: 0.4742281999997431
Update 4962 : Outer loss: 0.021751241758465767   time: 0.4847999000003256
Update 4963 : Outer loss: 0.019122684374451637   time: 0.472391200000402
Update 4964 : Outer loss: 0.04386021941900253   time: 0.4623833000000559
Update 4965 : Outer loss: 0.014112558215856552   time: 0.4672126999998909
Update 4966 : Outer loss: 0.01690530776977539   time: 0.4787370000003648
Update 4967 : Outer loss: 0.07033713907003403   time: 0.4836568000000625
Update 4968 : Outer loss: 0.020452970638871193   time: 0.47835189999977956
Update 4969 : Outer loss: 0.021763168275356293   time: 0.48902829999997266
Update 4970 : Outer loss: 0.016805391758680344   time: 0.48067829999990863
Update 4971 : Outer loss: 0.022874211892